pobieranie danych przez API kaggla

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
#print(os.listdir("../input"))
import argparse
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate
import tensorflow.keras.layers as layers

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import keras
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19

In [26]:
from tensorflow import keras
import tensorflow as tf
import argparse
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"paulinacieslinska","key":"8a9f1fe117227a73bb15099caabdc801"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
import os
import kaggle

def download_df():
    if not os.path.exists("data"):
        os.makedirs("data")
    k = kaggle.KaggleApi({"username": "jdsz5paulina", "key": "5277445bf2e6cef9aac564d8f7c5b87d"})
    k.authenticate()
    print("kaggle.com: authenticated")
    k.dataset_download_cli("grassknoted/asl-alphabet", unzip=True, path="data")
if __name__ == "__main__":
    download_df()

kaggle.com: authenticated


  0%|          | 0.00/1.03G [00:00<?, ?B/s]

100%|██████████| 1.03G/1.03G [00:11<00:00, 96.9MB/s]


In [7]:
test_data='/content/data/asl_alphabet_test/asl_alphabet_test'

In [8]:
train_data='/content/data/asl_alphabet_train/asl_alphabet_train'

In [9]:
target_size = (200,200)
target_dims = (200,200, 3) # add channel for RGB
n_classes = 29
val_frac = 0.1
batch_size = 64

data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_frac)

train_generator = data_augmentor.flow_from_directory(train_data, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(train_data, target_size=target_size, batch_size=batch_size, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [10]:
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}

def load_data():
    """
    Loads data and preprocess. Returns train and test data along with labels.
    """
    images = []
    labels = []
    size = 64,64
    print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_data):
        print(folder, end = ' | ')
        for image in os.listdir(train_data + "/" + folder):
            temp_img = cv2.imread(train_data + '/' + folder + '/' + image)
            temp_img = cv2.resize(temp_img, size)
            images.append(temp_img)
            labels.append(labels_dict[folder])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels)
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.05)
    
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
load_data()

tworzenie modelu

In [12]:
IMG_SIZE = (200, 200)
IN_SHAPE = (*IMG_SIZE, 3)

def get_model(pretrained_model, all_character_names):
    if pretrained_model == 'inception':
        pretrained_model = InceptionV3(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'xception':
        pretrained_model = Xception(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'resnet50':
        pretrained_model = ResNet50(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'vgg19':
        pretrained_model = VGG19(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'all':
        input = Input(shape=IN_SHAPE)
        inception_model = InceptionV3(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        xception_model = Xception(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        resnet_model = ResNet50(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        flattened_outputs = [Flatten()(inception_model.output),
                             Flatten()(xception_model.output),
                             Flatten()(resnet_model.output)]
        output = Concatenate()(flattened_outputs)
        pretrained_model = Model(input, output)
        if pretrained_model.output.shape.ndims > 2:
            output = Flatten()(pretrained_model.output)
        else:
            output = pretrained_model.output

        output = BatchNormalization()(output)
        output = Dropout(0.5)(output)
        output = Dense(128, activation='relu')(output)
        output = BatchNormalization()(output)
        output = Dropout(0.5)(output)
        output = Dense(len(all_character_names), activation='softmax')(output)
        model = Model(pretrained_model.input, output)
        for layer in pretrained_model.layers:
            layer.trainable = False
        model.summary(line_length=200)

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

In [13]:
predicted_labels = model.predict(pixels, batch_size=1)

NameError: ignored

tworzenie własnego modelu od podstaw

In [16]:
target_size=(200,200)

In [15]:
from keras import backend as K
K.set_image_data_format('channels_first')

In [19]:
def own_neural_net():
    s = Input((*target_size,3))
    c1 = Conv2D(8, (3,3), activation='relu', padding='same', name = 'c11') (s)
    c1 = Conv2D(8,(3,3), activation='relu', padding='same', name = 'c12') (c1)
    p1 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c1)

    c2 = Conv2D(32, (3,3), activation='relu', padding='same', name = 'c21') (p1)
    c2 = Conv2D(32, (3,3), activation='relu', padding='same', name = 'c22') (c2)
    p2 = MaxPooling2D(pool_size=(2,2),strides=(2,2)) (c2)

    # c3 = Conv2D(64,(3,3), activation='relu', padding='same', name = 'c31') (p2)
    # c3 = Conv2D(64, (3,3), activation='relu', padding='same', name = 'c32') (c3)
    # p3 = MaxPooling2D() (c3)
    # c4 = Conv2D(128, (3,3), activation='relu', padding='same', name = 'c41') (p3)
    # c4 = Conv2D(128, (3,3), activation='relu', padding='same', name = 'c42') (c4)
    # p4 = MaxPooling2D() (c4)
    # c5 = Conv2D(256, (3,3), activation='relu', padding='same', name = 'c51') (p4)
    # c5 = Conv2D(256, (3,3), activation='relu', padding='same', name = 'c52') (c5)


    f_end=keras.layers.Flatten()(p2)

    z=Dense(units=4096, activation='tanh')(f5)
    z=Dense(units=4096, activation='tanh')(z)
    out=keras.layers.Dense(units=1, activation = 'softmax')(z)



    return keras.models.Model(inputs=s,outputs=out)
    

In [27]:
model = own_neural_net()
model.compile(optimizer = 'adam', loss = 'acc')
model.summary()

ValueError: ignored

model nr 2

In [28]:
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3)
N_CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64

In [29]:
inp = Input(shape=TARGET_DIMS)
x1= Conv2D(16 ,(3,3),padding='same', activation='relu')(inp)
x1= Conv2D(32,(3,3),padding='same', activation='relu')(x1)
x1= MaxPooling2D(2,2)(x1)
x2= Conv2D(32,(3,3),padding='same', activation='relu')(x1)
x2= Conv2D(64,(3,3),padding='same', activation='relu')(x2)
x2= MaxPooling2D(3,3)(x2)
x3= Conv2D(64,(3,3),padding='same', activation='relu')(x2)
x3= Conv2D(64,(2,2),padding='same', activation='relu')(x3)
x3= Conv2D(64,(3,3),padding='same', activation='relu')(x3)
x3= Dropout(0.5)(x3)
x3= MaxPooling2D(2,2,padding='same')(x3)
x4= Conv2D(64,(3,3),padding='same', activation='relu')(x3)
x4= MaxPooling2D(2,2,padding='same')(x4)
x = Dropout(0.5)(x4)
o = Concatenate(axis=3)([x1, x2, x3, x4,x])
y = Flatten()(o)
y=  Dropout(0.2)(y)
y = Dense(200, activation='relu')(y)
#y=  Dropout(0.2)(y)
y = Dense(N_CLASSES, activation='softmax')(y) 

model = Model(inp, y)
model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

ValueError: ignored

model 3